# 1. Import library

In [78]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data


import time
import numpy as np
import pickle
import copy
import pandas as pd
# from AttentiveLayers_Viz import *
from Featurizer import *
from getFeatures import *

# Attentive Layers

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda:1')

class Fingerprint_viz(nn.Module):
    def __init__(self, radius, T, input_feature_dim, input_bond_dim,
                 fingerprint_dim, output_units_num, p_dropout):
        super(Fingerprint_viz, self).__init__()

        # Graph attention for atom embedding
        self.atom_fc = nn.Linear(input_feature_dim, fingerprint_dim)
        self.neighbor_fc = nn.Linear(input_feature_dim+input_bond_dim, fingerprint_dim)
        # self.GRUCell = nn.ModuleList([nn.GRUCell(fingerprint_dim, fingerprint_dim) for r in range(radius)])
        self.align = nn.ModuleList([nn.Linear(2*fingerprint_dim, 1) for r in range(radius)])
        self.attend = nn.ModuleList([nn.Linear(fingerprint_dim, fingerprint_dim) for r in range(radius)])

        # Graph attention for molecule embedding
        self.mol_GRUCell = nn.GRUCell(fingerprint_dim, fingerprint_dim)
        self.mol_align = nn.Linear(2*fingerprint_dim, 1)
        self.mol_attend = nn.Linear(fingerprint_dim, fingerprint_dim)

        self.dropout = nn.Dropout(p=p_dropout)
        self.output = nn.Linear(fingerprint_dim, output_units_num)

        self.radius = radius
        self.T = T

    def forward(self, atom_list, bond_list, atom_degree_list, bond_degree_list, atom_mask):
        atom_list = atom_list.to(device)
        bond_list = bond_list.to(device)
        atom_degree_list = atom_degree_list.to(device)
        bond_degree_list = bond_degree_list.to(device)
        atom_mask = atom_mask.to(device)
        
        atom_mask = atom_mask.unsqueeze(2)
        batch_size, mol_length, num_atom_feat = atom_list.size()
        atom_feature = F.relu(self.atom_fc(atom_list))
        
        atom_feature_viz = []
        atom_feature_viz.append(self.atom_fc(atom_list))
                                
        bond_neighbor = [bond_list[i][bond_degree_list[i]] for i in range(batch_size)]
        bond_neighbor = torch.stack(bond_neighbor, dim=0)
        atom_neighbor = [atom_list[i][atom_degree_list[i]] for i in range(batch_size)]
        atom_neighbor = torch.stack(atom_neighbor, dim=0)

        # Concatenate atom and bond features
        neighbor_feature = torch.cat([atom_neighbor, bond_neighbor], dim=-1)
        neighbor_feature = F.relu(self.neighbor_fc(neighbor_feature))

        # Generate mask to eliminate the influence of blank atoms
        attend_mask = atom_degree_list.clone()
        attend_mask[attend_mask != mol_length-1] = 1
        attend_mask[attend_mask == mol_length-1] = 0
        attend_mask = attend_mask.type(torch.FloatTensor).unsqueeze(-1).to(device)

        softmax_mask = atom_degree_list.clone()
        softmax_mask[softmax_mask != mol_length-1] = 0
        softmax_mask[softmax_mask == mol_length-1] = -9
        softmax_mask = softmax_mask.type(torch.FloatTensor).unsqueeze(-1).to(device)

        batch_size, mol_length, max_neighbor_num, fingerprint_dim = neighbor_feature.shape
        atom_feature_expand = atom_feature.unsqueeze(-2).expand(batch_size, mol_length, max_neighbor_num, fingerprint_dim)
        feature_align = torch.cat([atom_feature_expand, neighbor_feature], dim=-1)

        align_score = F.relu(self.align[0](self.dropout(feature_align)))
        align_score = align_score + softmax_mask
        attention_weight = F.softmax(align_score, -2)
        attention_weight = attention_weight * attend_mask
        
        atom_attention_weight_viz = []
        atom_attention_weight_viz.append(attention_weight)
                                
                                
                                
        neighbor_feature_transform = self.attend[0](self.dropout(neighbor_feature))
        context = torch.sum(torch.mul(attention_weight, neighbor_feature_transform), -2)
        context = F.relu(context)
        context_reshape = context.view(batch_size * mol_length, fingerprint_dim)
        atom_feature_reshape = atom_feature.view(batch_size * mol_length, fingerprint_dim)
        # atom_feature_reshape = self.GRUCell[0](context_reshape, atom_feature_reshape)
        atom_feature = atom_feature_reshape.view(batch_size, mol_length, fingerprint_dim)

        activated_features = F.relu(atom_feature)
        atom_feature_viz.append(activated_features)
                                
                                
        for d in range(1, self.radius):
            neighbor_feature = [activated_features[i][atom_degree_list[i]] for i in range(batch_size)]
            neighbor_feature = torch.stack(neighbor_feature, dim=0)
            atom_feature_expand = activated_features.unsqueeze(-2).expand(batch_size, mol_length, max_neighbor_num, fingerprint_dim)

            feature_align = torch.cat([atom_feature_expand, neighbor_feature], dim=-1)

            align_score = F.relu(self.align[d](self.dropout(feature_align)))
            align_score = align_score + softmax_mask
            attention_weight = F.softmax(align_score, -2)
            attention_weight = attention_weight * attend_mask
            
            atom_attention_weight_viz.append(attention_weight)
                                    
            neighbor_feature_transform = self.attend[d](self.dropout(neighbor_feature))
            context = torch.sum(torch.mul(attention_weight, neighbor_feature_transform), -2)
            context = F.relu(context)
            context_reshape = context.view(batch_size * mol_length, fingerprint_dim)
            # atom_feature_reshape = self.GRUCell[d](context_reshape, atom_feature_reshape)
            atom_feature = atom_feature_reshape.view(batch_size, mol_length, fingerprint_dim)

            activated_features = F.relu(atom_feature)
            atom_feature_viz.append(activated_features)                    
            
        mol_feature_unbounded_viz = []
        mol_feature_unbounded_viz.append(torch.sum(atom_feature * atom_mask, dim = -2))
                                
        mol_feature = torch.sum(activated_features * atom_mask, dim=-2)
        activated_features_mol = F.relu(mol_feature)
        
        mol_feature_viz = []
        mol_feature_viz.append(mol_feature)
        
                                
        mol_attention_weight_viz = []
        mol_softmax_mask = atom_mask.clone()
        mol_softmax_mask[mol_softmax_mask == 0] = -9e8
        mol_softmax_mask[mol_softmax_mask == 1] = 0
        mol_softmax_mask = mol_softmax_mask.type(torch.FloatTensor).to(device)

        for t in range(self.T):
            mol_prediction_expand = activated_features_mol.unsqueeze(-2).expand(batch_size, mol_length, fingerprint_dim)
            mol_align = torch.cat([mol_prediction_expand, activated_features], dim=-1)
            mol_align_score = F.relu(self.mol_align(mol_align))
            mol_align_score = mol_align_score + mol_softmax_mask
            mol_attention_weight = F.softmax(mol_align_score, -2)
            mol_attention_weight = mol_attention_weight * atom_mask
            mol_attention_weight_viz.append(mol_attention_weight)
                                
            activated_features_transform = self.mol_attend(self.dropout(activated_features))
            mol_context = torch.sum(torch.mul(mol_attention_weight, activated_features_transform), -2)
            mol_context = F.relu(mol_context)
            mol_feature = self.mol_GRUCell(mol_context, mol_feature)
            mol_feature_unbounded_viz.append(mol_feature)

            activated_features_mol = F.relu(mol_feature)
            mol_feature_viz.append(activated_features_mol)

        mol_prediction = mol_feature

        return atom_feature, atom_feature_viz, atom_attention_weight_viz, mol_feature_viz, mol_feature_unbounded_viz, mol_attention_weight_viz, mol_prediction

        
        
class ExtendedFingerprint_viz(Fingerprint_viz):
    def __init__(self, radius, T, input_feature_dim, input_bond_dim, fingerprint_dim, output_units_num, p_dropout, physicochemical_feature_dim, physicochemical_feature_dim_1, physicochemical_feature_dim_2, final1_fc1, final1_fc2):
 
        
        super().__init__(radius, T, input_feature_dim, input_bond_dim, fingerprint_dim, output_units_num, p_dropout)
        self.physicochemical_fc = nn.Linear(physicochemical_feature_dim, physicochemical_feature_dim_1) # physicochemical_feature_dim_1
        self.physicochemical_bn = nn.BatchNorm1d(physicochemical_feature_dim_1)
        
        self.physicochemical_fc2 = nn.Linear(physicochemical_feature_dim_1, physicochemical_feature_dim_2) # physicochemical_feature_dim_2
        self.physicochemical_bn2 = nn.BatchNorm1d(physicochemical_feature_dim_2)
        
        self.physicochemical_fc3 = nn.Linear(physicochemical_feature_dim_2, fingerprint_dim)
        self.physicochemical_bn3 = nn.BatchNorm1d(fingerprint_dim)
        
        self.final1_fc = nn.Linear(fingerprint_dim * 2, final1_fc1) # final1_fc
        self.bn_final1 = nn.BatchNorm1d(final1_fc1) # Added BatchNorm Layer

        self.final2_fc = nn.Linear(final1_fc1, final1_fc2) # final2_bc
        self.bn_final2 = nn.BatchNorm1d(final1_fc2) # Added BatchNorm Layer
        self.final3_fc = nn.Linear(final1_fc2, output_units_num)

    def forward(self, atom_list, bond_list, atom_degree_list, bond_degree_list, atom_mask, physicochemical_features):

        physicochemical_features = physicochemical_features.to(device)

        atom_feature, atom_feature_viz, atom_attention_weight_viz, mol_feature_viz, mol_feature_unbounded_viz, mol_attention_weight_viz, mol_prediction = super().forward(atom_list, bond_list, atom_degree_list, bond_degree_list, atom_mask)
        
        processed_physicochemical_features = F.relu(self.physicochemical_bn(self.physicochemical_fc(physicochemical_features)))
        processed_physicochemical_features2 = self.dropout(processed_physicochemical_features)
        processed_physicochemical_features3 = F.relu(self.physicochemical_bn2(self.physicochemical_fc2(processed_physicochemical_features2)))
        processed_physicochemical_features4 = self.dropout(processed_physicochemical_features3)
        processed_physicochemical_features5 = F.relu(self.physicochemical_bn3(self.physicochemical_fc3(processed_physicochemical_features4)))
        processed_physicochemical_features6 = self.dropout(processed_physicochemical_features5)
        
        
        combined_feature_vector = torch.cat([mol_prediction, processed_physicochemical_features6], dim=-1)
        
        #fingerprint_dim => 1024
        final_prediction = F.relu(self.bn_final1(self.final1_fc(combined_feature_vector))) # Applied BatchNorm Layer
        final2_prediction = self.dropout(final_prediction)
        final3_prediction = F.relu(self.bn_final2(self.final2_fc(final2_prediction))) # Applied BatchNorm Layer
        final4_prediction = self.dropout(final3_prediction)
        final5_prediction = self.final3_fc(final4_prediction)
        final5_prediction = F.softmax(final5_prediction, dim = 1)

       
      
        return atom_feature, atom_feature_viz, atom_attention_weight_viz, mol_feature_viz, mol_feature_unbounded_viz, mol_attention_weight_viz, final5_prediction



In [4]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

In [5]:
from rdkit import Chem
from rdkit.Chem import QED
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from IPython.display import display, SVG
import seaborn as sns

# 2. Data load and preparing

In [6]:
import os
os.chdir('/data/home/ldhyun7222/hERGAT_유전독성 평가/dataset')

In [7]:
task_name = 'Class'
tasks = ['Class']
raw_filename = "ames_cls_final_data.csv"
feature_filename = raw_filename.replace('.csv', 'pickle')
filename = raw_filename.replace('csv', '')
prefix_filename = raw_filename.split('/')[-1].replace('.csv', '')

In [8]:
smiles_tasks_df = pd.read_csv(raw_filename)
smilesList = smiles_tasks_df.SMILES.values
print('number of all smiles :', len(smilesList))

number of all smiles : 9139


### Canonical smiles 생성 및 중복된 값 삭제

In [ ]:
atom_num_dist = []
remained_smiles = []
canonical_smiles_list = []
for smiles in smilesList:
    try:
        mol = Chem.MolFromSmiles(smiles)
        atom_num_dist.append(len(mol.GetAtoms()))
        remained_smiles.append(smiles)
        canonical_smiles_list.append(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles = True))
    except:
        print('not successfully processed smiles : ', smiles)
        pass
print('number of successfully processed smiles : ', len(remained_smiles))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 152)

In [10]:
smiles_tasks_df = smiles_tasks_df[smiles_tasks_df['SMILES'].isin(remained_smiles)]
smiles_tasks_df['cano_smiles'] = canonical_smiles_list
smiles_tasks_df.head()

,SMILES,Class,cano_smiles
0,Br/C=C/Br,1,Br/C=C/Br
1,BrC(Br)Br,1,BrC(Br)Br
2,BrC(Br)C(Br)(Br)Br,0,BrC(Br)C(Br)(Br)Br
3,BrC(Br)C(Br)Br,0,BrC(Br)C(Br)Br
4,BrC/C=C\CBr,1,BrC/C=C\CBr


In [11]:
random_seed = 100
radius = 3
T = 2
output_units_num = 2
per_task_output_units_num = 2

In [12]:
# Use the length of SMILES less than 100

smilesList = [smiles for smiles in canonical_smiles_list if len(Chem.MolFromSmiles(smiles).GetAtoms()) < 101]
uncovered = [smiles for smiles in canonical_smiles_list if len(Chem.MolFromSmiles(smiles).GetAtoms()) > 100]
smiles_tasks_df = smiles_tasks_df[~smiles_tasks_df['cano_smiles'].isin(uncovered)]
feature_dicts=get_smiles_dicts(smilesList)

P
[Na+]


In [13]:
remained_df = smiles_tasks_df[smiles_tasks_df['cano_smiles'].isin(feature_dicts['smiles_to_atom_mask'].keys())]
uncovered_df = smiles_tasks_df.drop(remained_df.index)
remained_df.reset_index()
# remained_df.to_csv('remained_df.csv', index = False)

remained_df

,SMILES,Class,cano_smiles
0,Br/C=C/Br,1,Br/C=C/Br
1,BrC(Br)Br,1,BrC(Br)Br
2,BrC(Br)C(Br)(Br)Br,0,BrC(Br)C(Br)(Br)Br
3,BrC(Br)C(Br)Br,0,BrC(Br)C(Br)Br
4,BrC/C=C\CBr,1,BrC/C=C\CBr
...,...,...,...
9134,c1csc(-c2nc(N3CCOCC3)c3ccccc3n2)c1,1,c1csc(-c2nc(N3CCOCC3)c3ccccc3n2)c1
9135,c1cscn1,1,c1cscn1
9136,c1nc[nH]n1,0,c1nc[nH]n1
9137,c1scc2c1-c1cscc1C1NC21,1,c1scc2c1-c1cscc1C1NC21


### physicochemical properties에 의해 생성된 new_remained_df

In [14]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import rdMolDescriptors

In [15]:
remained_df = remained_df.reset_index()

# mol 형태로 변환
train_mols = [Chem.MolFromSmiles(smiles) for smiles in remained_df["cano_smiles"]]

# mol 형태로 변환이 되지 않은 경우, none_list에 담는다
none_list = []
for i in range(len(train_mols)):
    if train_mols[i] is None :
        none_list.append(i)
        print('none_list에 추가됨')
    
reg_idx = 0
for i in none_list :
    del train_mols[i - reg_idx]
    reg_idx += 1
    
# none_list가 존재할 경우, 삭제 후 데이터프레임 인덱스 맞춰주기
if len(none_list) != 0 :
    remained_df = remained_df.drop(none_list, axis=0)
    remained_df = remained_df.reset_index(drop = True)

# fingerprint 생성
bit_info_list = [] # bit vector의 설명자 리스트 담기
bit_info = {} #bit vector 설명자
fps = []

b = 0
# mol 파일에서 fingerprint Bit Vector 형태로 변환하기
for a in train_mols :
    fps.append(AllChem.GetMorganFingerprintAsBitVect(a, 3, nBits = 1024, bitInfo = bit_info))
    bit_info_list.append(bit_info.copy()) # bit_info 그대로 가져오면 변수가 변해서 리스트 값이 달라지므로 .copy()
    
# array 변환

arr_list = []
for i in range(len(fps)):
    array = np.zeros((0,), dtype = np.int8)
    arr_list.append(array)
for i in range(len(fps)):
    bit = fps[i]
    DataStructs.ConvertToNumpyArray(bit, arr_list[i])
    
train_x = np.stack([i.tolist() for i in arr_list])
train_finprt = pd.DataFrame(train_x)
train_finprt.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
import joblib
from sklearn.preprocessing import StandardScaler

# StandardScaler

sds_scaler = StandardScaler()

# molecular physicochemical properties 구하기

from rdkit.Chem import QED

train_qe = [QED.properties(mol) for mol in train_mols]
train_qe = pd.DataFrame(train_qe)
train_qe = train_qe.drop(columns = ['ROTB', 'AROM', 'ALERTS'])
train_qe[['MW', 'ALOGP', 'HBA', 'HBD', 'PSA']] = sds_scaler.fit_transform(train_qe[['MW', 'ALOGP', 'HBA', 'HBD', 'PSA']])
input_df=pd.concat([train_finprt, train_qe], axis=1)
new_remained_df = pd.concat([remained_df, input_df], axis = 1)
new_remained_df = new_remained_df.drop(columns = ['index'])
new_remained_df

,SMILES,Class,cano_smiles,0,1,2,3,4,5,6,...,1019,1020,1021,1022,1023,MW,ALOGP,HBA,HBD,PSA
0,Br/C=C/Br,1,Br/C=C/Br,0,0,0,0,0,0,0,...,0,0,0,0,0,-0.614821,-0.165763,-1.341869,-0.812407,-1.323096
1,BrC(Br)Br,1,BrC(Br)Br,0,1,0,0,0,0,0,...,0,0,0,0,0,-0.095414,-0.064193,-1.341869,-0.812407,-1.323096
2,BrC(Br)C(Br)(Br)Br,0,BrC(Br)C(Br)(Br)Br,0,1,0,0,0,0,0,...,0,0,0,0,0,1.238876,0.663940,-1.341869,-0.812407,-1.323096
3,BrC(Br)C(Br)Br,0,BrC(Br)C(Br)Br,0,1,0,0,0,0,0,...,0,0,0,0,0,0.626196,0.309893,-1.341869,-0.812407,-1.323096
4,BrC/C=C\CBr,1,BrC/C=C\CBr,0,0,0,0,0,0,0,...,0,0,0,0,0,-0.396963,-0.124116,-1.341869,-0.812407,-1.323096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9131,c1csc(-c2nc(N3CCOCC3)c3ccccc3n2)c1,1,c1csc(-c2nc(N3CCOCC3)c3ccccc3n2)c1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.251339,0.298477,0.219877,-0.812407,-0.490945
9132,c1cscn1,1,c1cscn1,0,0,0,0,0,0,0,...,0,0,0,0,0,-1.396941,-0.706828,-0.951432,-0.812407,-1.042667
9133,c1nc[nH]n1,0,c1nc[nH]n1,0,0,0,0,0,0,0,...,0,1,0,0,0,-1.521689,-1.362594,-0.560996,-0.109086,-0.418717
9134,c1scc2c1-c1cscc1C1NC21,1,c1scc2c1-c1cscc1C1NC21,0,0,0,1,0,0,0,...,1,0,0,0,0,-0.463693,0.289020,-0.951432,-0.109086,-0.845779


In [17]:
for key, value in feature_dicts.items():
    print(key)

smiles_to_atom_mask
smiles_to_atom_info
smiles_to_bond_info
smiles_to_atom_neighbors
smiles_to_bond_neighbors
smiles_to_rdkit_list


가중치는 전체 샘플에서 해당 클래스의 샘플 수로 결정하여 소수 클래스에 더 높은 가중치를 부여하게 한다...


crossentropyloss와 같이 가중치를 사용하면 클래스 불균형 문제를 해결할 수 있다.

- 소수 클래스의손실이 전체 손실에 더 영향을 많이 주게 되어 소수 클래스에도 더예측할 수 있도록 학습하게 하는 효과

In [18]:
weights = []

for i, task in enumerate(['Class']):
    negative_df = new_remained_df[new_remained_df['Class'] == 0][['SMILES', 'Class']]
    positive_df = new_remained_df[new_remained_df['Class'] == 1][['SMILES', 'Class']]

    weights.append([(positive_df.shape[0] + negative_df.shape[0]) / negative_df.shape[0],
    (positive_df.shape[0] + negative_df.shape[0]) / positive_df.shape[0]])    

In [19]:
print(positive_df.shape)
print(negative_df.shape)


(4429, 2)
(4707, 2)


In [20]:
# weights = torch.tensor(weights[0])
print('weights:', weights)

weights: [[1.940939026981092, 2.0627681192142697]]


In [21]:
weights

[[1.940939026981092, 2.0627681192142697]]

 이렇게하게되면 positive나 negative한 blocker가 데이터의 양이 다르기 때문에     
데이터 불균형 문제를 해결해주기 위해서 학습할때의 weights를 이용하여 loss function에 적용하게 됨

 - 한마디로 모델이, 샘플수가 더 적은곳에 더 많은 관심을 기울여 중요하게 여김...

In [22]:
test_df = new_remained_df.sample(frac = 1/10, random_state = 100)
train_df = new_remained_df.drop(test_df.index)  

train_df = train_df.reset_index(drop = True)  # (1737, 1032)
test_df = test_df.reset_index(drop = True)    # (793, 1032)

In [23]:
print('train_df label의 shape')
print(train_df['Class'].value_counts())
print('=========================')
print('test_df label의 shape')
print(test_df['Class'].value_counts())


train_df label의 shape
0    4277
1    3945
Name: Class, dtype: int64
test_df label의 shape
1    484
0    430
Name: Class, dtype: int64


## GNN 사용하기 위한 데이터 준비

In [24]:
x_atom, x_bonds, x_atom_index,  x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array([smilesList[0]], feature_dicts)

num_atom_features = 39  # x_atom.shape[-1] 
num_bond_features = 10   # x_bonds.shape[-1]

# loss function에 weioghts를 적용하여 더 크게 or 더 작게 학습할 가중치 설정
loss_function = [nn.CrossEntropyLoss(weight = torch.Tensor(weight).to(device), reduction = 'mean') for weight in weights]



### Bayesian optimizer로 찾은 최적의 parameter values


batch_size = 116
p_dropout = 0.54
fingerprint_dim = 101
learning_rate = 4.63
weight_decay = 2.08
radius = 3
T = 2{'target': 0.8686914087581867, 'params': {'T': 2.0201586404751994, 'batch_size': 217.348955103839, 'dropout': 0.2756774855452565, 'fingerprint_dim': 151.57188184190773, 'learning_rate_exp': 4.2769832938074135, 'radius': 1.1811575589535743, 'weight_decay_exp': 4.228435163808071}}

In [25]:
physicochemical_feature_dim = 1029



p_dropout = 0.217
batch_size = 116
final1_fc1 = 280
final1_fc2 = 100
fingerprint_dim = 276
learning_rate = 4.64
physicochemical_feature_dim_1 = 472
physicochemical_feature_dim_2 = 249
weight_decay = 2.67


# batch_size = 230
# p_dropout = 0.2425
# fingerprint_dim = 100
# learning_rate = 4.63
# weight_decay = 2.51
# radius = 3
# T = 3

model fingerprint_viz 선언

In [26]:
model = ExtendedFingerprint_viz(radius, T, num_atom_features, num_bond_features, fingerprint_dim, output_units_num, p_dropout, physicochemical_feature_dim, physicochemical_feature_dim_1, physicochemical_feature_dim_2, final1_fc1, final1_fc2)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr = 10**-learning_rate, weight_decay = 10**-weight_decay)

model_parameters = filter(lambda p : p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

1650959


In [27]:
model

ExtendedFingerprint_viz(
  (atom_fc): Linear(in_features=39, out_features=276, bias=True)
  (neighbor_fc): Linear(in_features=49, out_features=276, bias=True)
  (align): ModuleList(
    (0): Linear(in_features=552, out_features=1, bias=True)
    (1): Linear(in_features=552, out_features=1, bias=True)
    (2): Linear(in_features=552, out_features=1, bias=True)
  )
  (attend): ModuleList(
    (0): Linear(in_features=276, out_features=276, bias=True)
    (1): Linear(in_features=276, out_features=276, bias=True)
    (2): Linear(in_features=276, out_features=276, bias=True)
  )
  (mol_GRUCell): GRUCell(276, 276)
  (mol_align): Linear(in_features=552, out_features=1, bias=True)
  (mol_attend): Linear(in_features=276, out_features=276, bias=True)
  (dropout): Dropout(p=0.217, inplace=False)
  (output): Linear(in_features=276, out_features=2, bias=True)
  (physicochemical_fc): Linear(in_features=1029, out_features=472, bias=True)
  (physicochemical_bn): BatchNorm1d(472, eps=1e-05, momentum=0.1

In [28]:
for name, param in model.named_parameters():
    print(name, param.data.shape)

atom_fc.weight torch.Size([276, 39])
atom_fc.bias torch.Size([276])
neighbor_fc.weight torch.Size([276, 49])
neighbor_fc.bias torch.Size([276])
align.0.weight torch.Size([1, 552])
align.0.bias torch.Size([1])
align.1.weight torch.Size([1, 552])
align.1.bias torch.Size([1])
align.2.weight torch.Size([1, 552])
align.2.bias torch.Size([1])
attend.0.weight torch.Size([276, 276])
attend.0.bias torch.Size([276])
attend.1.weight torch.Size([276, 276])
attend.1.bias torch.Size([276])
attend.2.weight torch.Size([276, 276])
attend.2.bias torch.Size([276])
mol_GRUCell.weight_ih torch.Size([828, 276])
mol_GRUCell.weight_hh torch.Size([828, 276])
mol_GRUCell.bias_ih torch.Size([828])
mol_GRUCell.bias_hh torch.Size([828])
mol_align.weight torch.Size([1, 552])
mol_align.bias torch.Size([1])
mol_attend.weight torch.Size([276, 276])
mol_attend.bias torch.Size([276])
output.weight torch.Size([2, 276])
output.bias torch.Size([2])
physicochemical_fc.weight torch.Size([472, 1029])
physicochemical_fc.bias t

In [29]:
def train(model, dataset, optimizer, loss_function):
    model.train()
    np.random.seed(30)
    valList = np.arange(0,dataset.shape[0])
    #shuffle them
    np.random.shuffle(valList)
    batch_list = []
    for i in range(0, dataset.shape[0], batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch)   
    for counter, train_batch in enumerate(batch_list):
        batch_df = dataset.loc[train_batch,:]
        smiles_list = batch_df.cano_smiles.values
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)    
        x_atom = torch.Tensor(x_atom).to(device).float()
        x_bonds = torch.Tensor(x_bonds).to(device).float()
        x_atom_index = torch.LongTensor(x_atom_index).to(device)
        x_bond_index = torch.LongTensor(x_bond_index).to(device)
        x_mask = torch.Tensor(x_mask).to(device).float()
        
        selected_rows = batch_df[batch_df['cano_smiles'] == smiles_list].drop(columns=['SMILES', 'Class', 'cano_smiles'])
        physicochemical_features= torch.tensor(selected_rows.to_numpy(dtype=np.float32), dtype=torch.float).to(device)

        atoms_prediction, atom_feature_viz, atom_attention_weight_viz, mol_feature_viz, mol_feature_unbounded_viz, mol_attention_weight_viz, mol_prediction = model(x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, physicochemical_features)
        
        model.zero_grad()
        # Step 4. Compute your loss function. (Again, Torch wants the target wrapped in a variable)
        loss = 0.0
        for i,task in enumerate(tasks):
            y_pred = mol_prediction[:, i * per_task_output_units_num:(i + 1) *
                                    per_task_output_units_num].to(device)
            y_val = batch_df["Class"].values

            validInds = np.where((y_val == 0) | (y_val == 1))[0]
            if len(validInds) == 0:
                continue

            y_val_adjust = np.array([y_val[v] for v in validInds]).astype(float)
            y_val_adjust = torch.LongTensor(y_val_adjust).to(device)
            validInds = torch.LongTensor(validInds).squeeze().to(device)
            y_pred_adjust = torch.index_select(y_pred, 0, validInds)
            y_pred_adjust = y_pred_adjust.to(device)
            loss += loss_function[i](
                y_pred_adjust,y_val_adjust)
        # Step 5. Do the backward pass and update the gradient
#             print(y_val,y_pred,validInds,y_val_adjust,y_pred_adjust)
        loss.backward()
        optimizer.step()
        
        
def eval(model, dataset):
    model.eval()
    y_val_list = {}
    y_pred_list = {}
    losses_list = []

    
    valList = np.arange(0,dataset.shape[0])
    batch_list = []
    for i in range(0, dataset.shape[0], batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch)   
    for counter, test_batch in enumerate(batch_list):
        batch_df = dataset.loc[test_batch,:]
        smiles_list = batch_df.cano_smiles.values
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,feature_dicts)
        x_atom = torch.Tensor(x_atom).to(device).float()
        x_bonds = torch.Tensor(x_bonds).to(device).float()
        x_atom_index = torch.LongTensor(x_atom_index).to(device)
        x_bond_index = torch.LongTensor(x_bond_index).to(device)
        x_mask = torch.Tensor(x_mask).to(device).float()
        
        selected_rows = batch_df[batch_df['cano_smiles'] == smiles_list].drop(columns=['SMILES', 'Class', 'cano_smiles'])
        physicochemical_features = torch.tensor(selected_rows.to_numpy(dtype = np.float32), dtype = torch.float).to(device)
#        physicochemical_features = physicochemical_features.expand(batch_size, -1)
        
        atoms_prediction, atom_feature_viz, atom_attention_weight_viz, mol_feature_viz, mol_feature_unbounded_viz, mol_attention_weight_viz, mol_prediction = model(x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, physicochemical_features)

        atom_pred = atoms_prediction.data[:,:,1].unsqueeze(2).cpu().numpy()
        for i,task in enumerate(tasks):
            y_pred = mol_prediction[:, i * per_task_output_units_num:(i + 1) *
                                    per_task_output_units_num].to(device)
            y_val = batch_df[task].values

            validInds = np.where((y_val==0) | (y_val==1))[0]
            
            if len(validInds) == 0:
                continue
            y_val_adjust = np.array([y_val[v] for v in validInds]).astype(float)
            y_val_adjust = torch.LongTensor(y_val_adjust).to(device)
            validInds = torch.LongTensor(validInds).squeeze().to(device)
            y_pred_adjust = torch.index_select(y_pred, 0, validInds)
            y_pred_adjust = y_pred_adjust.to(device)

            loss = loss_function[i](
                y_pred_adjust,y_val_adjust)
            y_pred_adjust = F.softmax(y_pred_adjust,dim=-1).data.cpu().numpy()[:,1]
            losses_list.append(loss.detach().cpu().numpy())
            try:
                y_val_list[i].extend(y_val_adjust.cpu().numpy())
                y_pred_list[i].extend(y_pred_adjust)
            except:
                y_val_list[i] = []
                y_pred_list[i] = []
                y_val_list[i].extend(y_val_adjust.cpu().numpy())
                y_pred_list[i].extend(y_pred_adjust)
                
    # 시행할때마다 모델에서 최적의 optimal_threshold값을 찾아 성능평가
    
    def find_optimal_threshold(precision, recall, thresholds):
        with np.errstate(divide='ignore', invalid='ignore'):
            f1_scores = 2*((precision*recall)/(precision+recall))
            f1_scores = np.nan_to_num(f1_scores)  # convert NaNs to 0
        index = np.argmax(f1_scores)
        return thresholds[index], f1_scores[index]
    fpr_list, tpr_list, roc_auc_list, precision_list, recall_list, pr_auc_list = [], [], [], [], [], []    

    test_prc = []
    test_thresholds = []
    optimal_thresholds = []

    precision, recall, thresholds = precision_recall_curve(y_val_list[0], y_pred_list[0])
    optimal_threshold, test_f1_scores = find_optimal_threshold(precision, recall, thresholds)
    optimal_thresholds.append(optimal_threshold)
    pr_auc = auc(recall, precision)
    pr_auc_list.append(pr_auc)
    precision_list.append(precision)
    recall_list.append(recall)
    test_prc.append(auc(recall, precision))
    test_thresholds.append(thresholds)
   

    
    fpr, tpr, thresholds = roc_curve(y_val_list[0], y_pred_list[0])
    roc_auc = auc(fpr, tpr)
    fpr_list.append(fpr)
    tpr_list.append(tpr)
    roc_auc_list.append(roc_auc)


    test_acc = [accuracy_score(y_val_list[i], (np.array(y_pred_list[i]) > optimal_thresholds[i]).astype(int)) for i in range(len(tasks))]    
    test_roc = [roc_auc_score(y_val_list[i], y_pred_list[i]) for i in range(len(tasks))]
    test_loss = np.array(losses_list).mean()
    
    return test_acc, test_roc, test_prc, test_f1_scores, test_loss, fpr_list, tpr_list, roc_auc_list, precision_list, recall_list, pr_auc_list, optimal_thresholds, y_val_list, y_pred_list

In [30]:
best_params = {"roc_epoch": 0, "valid_acc": 0, "valid_roc": 0, "valid_precision": 0, "valid_recall": 0, "valid_f1_score": 0, "valid_loss": float('inf')}
best_params

{'roc_epoch': 0,
 'valid_acc': 0,
 'valid_roc': 0,
 'valid_precision': 0,
 'valid_recall': 0,
 'valid_f1_score': 0,
 'valid_loss': inf}

### Early Stopping

In [31]:
class EarlyStopping:
    def __init__(self, patience=5):
        self.loss = np.inf
        self.counter = 0
        self.patience = patience
        self.early_stop = False

    def step(self, loss):
        if loss < self.loss:
            self.loss = loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def should_step(self):
        return self.early_stop
    
early_stopping = EarlyStopping(patience = 15)    

### model training

In [32]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
fold_results = []
train_df = train_df.reset_index().drop(columns = 'index')
training_df = train_df.copy()


In [33]:
X = pd.concat([training_df, test_df], axis = 0)
X = X.reset_index()
Y = X['Class']
training_df2 = X.copy()
training_df2 = training_df2.drop(columns = 'index')
X = X.drop(columns = ['index', 'Class'])

In [34]:
import os
os.chdir('/data/home/ldhyun7222/hERGAT_유전독성 평가')
n_splits = 10
epochs = 300

valid_roc_list = []
valid_loss_vis2 = []

valid_f1_score_list = []
valid_acc_list = []
valid_aupr_list = []


tpr_list = []
fpr_list = []
roc_auc_list = []

precision_list = []
recall_list = []
pr_auc_list = []



train_roc2 = []
valid_roc2 = []
train_sensitivity2 = []
train_specificity2 = []
valid_precision2 = []
valid_recall2 = []
valid_prc2 = []
valid_optimal_thresholds2=[]

for fold, (train_index, test_index) in enumerate(skf.split(X, Y)):
    print('***********************'*4)
    print(f'Fold {fold + 1} / {n_splits}')
    train_df2 = training_df2.iloc[train_index].reset_index(drop = True)
    valid_df2 = training_df2.iloc[test_index].reset_index(drop = True)
    
    # 모델 초기화
    model = ExtendedFingerprint_viz(radius, T, num_atom_features, num_bond_features, fingerprint_dim, output_units_num, p_dropout, physicochemical_feature_dim, physicochemical_feature_dim_1, physicochemical_feature_dim_2, final1_fc1, final1_fc2)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr = 10**-learning_rate, weight_decay = 10**-weight_decay)

    # early stopping도 초기화
    early_stopping = EarlyStopping(patience = 15)
    
    # 각 fold별 최적 매개변수 초기화
    fold_best_params = best_params.copy()
    
    for epoch in range(epochs):    
        start_time = time.time() # epoch의 시작시간
        
        train(model, train_df2, optimizer, loss_function)
        # eval_result = eval(model, valid_df2)

        
        valid_acc, valid_roc, valid_prc, valid_f1_score, valid_loss, valid_fpr_list, valid_tpr_list, valid_roc_auc_list, valid_precision_list, valid_recall_list, valid_pr_auc_list, optimal_thresholds, y_val_list, y_pred_list= eval(model, valid_df2)
        valid_loss_vis2.append(valid_loss)
        valid_roc2.append(valid_roc)
        valid_roc_mean = np.array(valid_roc2).mean()
        valid_prc2.append(valid_prc)
        
        if valid_roc_mean > 0.70:
             torch.save(model, 'saved_model/model_'+prefix_filename+'_'+str(fold)+'_'+str(epoch)+'.pt')                
            
        if valid_roc_mean > fold_best_params["valid_roc"]:
            fold_best_params = {'roc_epoch': epoch, 
                               'valid_acc': valid_acc,
                               'valid_roc': valid_roc_mean,
                               'valid_f1_score': valid_f1_score,
                               'valid_aupr_score': valid_prc,
                               'valid_loss': valid_loss
                               }
               
        if valid_loss < fold_best_params["valid_loss"]:
            fold_best_params["valid_loss"] = valid_loss

        # fold_results.append(eval_result)

        print("EPOCH:\t"+str(epoch)+'\n'\
             +"valid_acc"+":"+str(valid_acc) + '\n'\
             +"valid_roc"+":"+str(valid_roc) + '\n'\
             
             +"valid_aupr"+":"+str(valid_prc) + '\n'\
             +"valid_f1_score"+":"+str(valid_f1_score))
        print('************************')
        early_stopping.step(valid_loss)
        
        if early_stopping.should_step():
            print('early stopping')
        
            break
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Epoch {epoch+1} finished in {elapsed_time:.2f} seconds.')

    fold_results.append(fold_best_params)
    valid_acc_list.append(valid_acc)
    valid_roc_list.append(valid_roc)
    valid_roc_mean = np.array(valid_roc).mean()
    valid_f1_score_list.append(valid_f1_score)
    valid_aupr_list.append(valid_prc)

    tpr_list.append(valid_tpr_list)
    fpr_list.append(valid_fpr_list)
    roc_auc_list.append(valid_roc_auc_list)
    precision_list.append(valid_precision_list)
    recall_list.append(valid_recall_list)
    pr_auc_list.append(valid_pr_auc_list)


final_result = {key: np.mean([result[key] for result in fold_results]) for key in fold_results[0].keys()}
print('cross_validation_results : ', final_result)



    

********************************************************************************************
Fold 1 / 10
EPOCH:	0
valid_acc:[0.6083150984682714]
valid_roc:[0.6953674282181421]
valid_aupr:[0.6925715901224895]
valid_f1_score:0.6751592356687898
************************
Epoch 1 finished in 2.95 seconds.
EPOCH:	1
valid_acc:[0.5842450765864332]
valid_roc:[0.7160453000915395]
valid_aupr:[0.7273750286520291]
valid_f1_score:0.6807076663858467
************************
Epoch 2 finished in 2.24 seconds.
EPOCH:	2
valid_acc:[0.6641137855579868]
valid_roc:[0.7351775435771353]
valid_aupr:[0.7432298213727148]
valid_f1_score:0.6877551020408164
************************
Epoch 3 finished in 2.24 seconds.
EPOCH:	3
valid_acc:[0.687089715536105]
valid_roc:[0.7576454687926846]
valid_aupr:[0.7612207513757399]
valid_f1_score:0.7076923076923077
************************
Epoch 4 finished in 2.07 seconds.
EPOCH:	4
valid_acc:[0.7024070021881839]
valid_roc:[0.7753686743061446]
valid_aupr:[0.7757848938449231]
valid_f1_

In [35]:
print('Accuracy : ', np.mean(valid_acc_list))
print('AUROC : ', np.mean(valid_roc_list))
print('AUPR : ', np.mean(valid_aupr_list))
print('F1-score : ', np.mean(valid_f1_score_list))

Accuracy :  0.8168787343525683
AUROC :  0.8859560278539552
AUPR :  0.8802333550174174
F1-score :  0.817101552481066


In [36]:
final_result

{'roc_epoch': 53.8,
 'valid_acc': 0.7744910016033899,
 'valid_roc': 0.8589908267619675,
 'valid_f1_score': 0.792445833248616,
 'valid_aupr_score': 0.8444720574024235,
 'valid_loss': 0.49179006}

# Stratified 10-cross validation AUROC 그래프 그리기

In [37]:
# ROC Curve 그리기
plt.figure()
for i in range(len(tpr_list)):
    plt.plot(fpr_list[i][0], tpr_list[i][0], lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.3f)' % (i+1, roc_auc_list[i][0]))

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig('in_vitro/Stratified hERGAT_AUROC_in_vitro.png')
plt.show()

In [38]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interp
from sklearn.metrics import auc

# 선형 보간을 위한 고른 간격의 FPR 값 설정
mean_fpr = np.linspace(0, 1, 100)

# 평균 및 표준편차 계산을 위한 배열 초기화
tpr_list_interp = []

# 각 fold의 ROC Curve에 대한 데이터 처리
for i in range(len(tpr_list)):
    # 선형 보간을 통해 모든 curve를 동일한 FPR 값에 맞춤
    interp_tpr = interp(mean_fpr, fpr_list[i][0], tpr_list[i][0])
    tpr_list_interp.append(interp_tpr)

# 평균 및 표준편차 계산
mean_tpr = np.mean(tpr_list_interp, axis=0)
std_tpr = np.std(tpr_list_interp, axis=0)

# 평균 ROC Curve 및 표준편차 영역 그리기
plt.figure()
plt.plot(mean_fpr, mean_tpr, color='b', label='Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (np.mean(roc_auc_list), np.mean(std_tpr)))
plt.fill_between(mean_fpr, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.3)

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig('in_vitro/Average_hERGAT_AUROC2_in_vitro.png')
plt.show()


/data/home/bmi-lab/anaconda3/envs/jy1/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
  from ipykernel import kernelapp as app


In [39]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy import interp

# # 모든 FPR 값들을 결합하여 고유한 FPR 값들의 배열을 생성
# all_fpr = np.unique(np.concatenate([fpr for fpr in fpr_list2]))

# # 이제 모든 FPR 값에 대해 TPR을 보간합니다.
# mean_tpr = np.zeros_like(all_fpr)
# std_tpr = np.zeros_like(all_fpr)
# for i in range(len(tpr_list2)):
#     mean_tpr += interp(all_fpr, fpr_list2[i], tpr_list2[i])

# mean_tpr /= len(tpr_list2)

# # 이제 표준편차를 계산합니다.
# for i in range(len(tpr_list2)):
#     std_tpr += (interp(all_fpr, fpr_list2[i], tpr_list2[i]) - mean_tpr)**2

# std_tpr = np.sqrt(std_tpr / len(tpr_list2))

# # 평균 ROC 곡선 및 표준편차 영역 그리기
# plt.figure()
# plt.plot(all_fpr, mean_tpr, color='b', label='hERGAT ROC (AUC = %0.3f ± %0.3f)' % (np.mean(roc_auc_list[0]), np.mean(std_tpr)))
# plt.fill_between(all_fpr, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2)




# plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=0.8)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve with Standard Deviation')
# plt.legend(loc="lower right")
# plt.savefig('in_vitro/Stratified hERGAT_AUROC_with_std_in_vitro.png')
# plt.show()

## strarified 10-cross validation의 AUPR 그리기

In [40]:
precision_list2 = [precision[0] for precision in precision_list]
recall_list2 = [recall[0] for recall in recall_list]
pr_auc_list2 = [pr[0] for pr in pr_auc_list]

# Precision-Recall Curve 그리기
plt.figure()
for i in range(len(precision_list2)):
    plt.plot(recall_list2[i], precision_list2[i], lw=2, alpha=0.3, label='PR fold %d (AUC = %0.3f)' % (i+1, pr_auc_list2[i]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.ylim([0.0, 1.0])  # Set the start of y-axis to 0
plt.savefig('in_vitro/Stratified hERGAT_AUPR_in_vitro.png')
plt.show()

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

# 각 fold의 recall 리스트를 동일한 길이로 보간
mean_recall = np.linspace(0, 1, 100)  # 균일한 recall 점을 정의
interp_precisions = []

for prec, rec in zip(precision_list2, recall_list2):
    # 각 fold에 대해 보간 함수 생성 및 적용
    interp_func = interpolate.interp1d(rec, prec, kind='linear', bounds_error=False, fill_value="extrapolate")
    interp_precision = interp_func(mean_recall)
    interp_precisions.append(interp_precision)

# 보간된 정밀도에 대한 평균 계산
mean_precision = np.mean(interp_precisions, axis=0)
std_precision = np.std(interp_precisions, axis=0)

# 평균 PR Curve 그리기
plt.figure()
plt.plot(mean_recall, mean_precision, color='b', label=r'hERGAT AUPR (AUC = %0.3f)' % np.mean(pr_auc_list[0]), lw=2, alpha=.8)
plt.fill_between(mean_recall, mean_precision - std_precision, mean_precision + std_precision, color='blue', alpha=0.2)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.savefig('in_vitro/Stratified hERGAT_AUPR_with_std_in_vitro.png')
plt.show()


In [ ]:
# np.savetxt('cross validation figure/Stratified 10-cross validation/hERGAT_all_fpr.csv', all_fpr, delimiter=',')
# np.savetxt('cross validation figure/Stratified 10-cross validation/hERGAT_mean_tpr.csv', mean_tpr, delimiter=',')
# np.savetxt('cross validation figure/Stratified 10-cross validation/hERGAT_std_tpr.csv', std_tpr, delimiter=',')
# np.savetxt('cross validation figure/Stratified 10-cross validation/hERGAT_roc_auc_list[0].csv', roc_auc_list[0], delimiter=',')

# np.savetxt('cross validation figure/Stratified 10-cross validation/hERGAT_mean_precision.csv', mean_precision, delimiter=',')
# np.savetxt('cross validation figure/Stratified 10-cross validation/hERGAT_mean_recall.csv', mean_recall, delimiter=',')


# np.savetxt('cross validation figure/Stratified 10-cross validation/hERGAT_std_precision.csv', std_precision, delimiter=',')
# np.savetxt('cross validation figure/Stratified 10-cross validation/hERGAT_pr_auc_list.csv', pr_auc_list[0], delimiter=',')

## loss_function 그리기

In [ ]:
def moving_average(data, window_size=5):
    """Calculate the moving average of given data."""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

# 원하는 창의 크기를 설정
window_size = 5  
smooth_train_loss = moving_average(train_loss_vis2, window_size)
smooth_valid_loss = moving_average(valid_loss_vis2, window_size)


In [ ]:
x_len_smooth = np.arange(len(smooth_train_loss)) + window_size // 2
fig, ax = plt.subplots()
ax.set_facecolor('white')  # Set the background color

plt.plot(x_len_smooth, smooth_train_loss, 'red', label='Train Loss')
plt.plot(x_len_smooth, smooth_valid_loss, 'blue', label='Valid Loss')

plt.legend(loc='upper right', facecolor = 'white')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.savefig('smooth_loss_function.png', facecolor = 'white')
plt.show()
